In [1]:
import pandas as pd
import os

directory = "../../Korpus"

In [2]:
df_wyborcza_new = pd.read_csv(f"{directory}/eco_wyborcza.csv")
df_wyborcza_old = pd.read_csv("eco_wyborcza.csv")

lemma = pd.read_csv("lemma.csv")

In [3]:
df_wyborcza = df_wyborcza_old.merge(df_wyborcza_new[['link', 'id', "eco_id"]], on='link', how='inner', suffixes=('', '_new'))

In [4]:
len(df_wyborcza_new) == len(df_wyborcza_old) == len(df_wyborcza)

True

In [5]:
lemma_wyborcza = lemma[lemma['source'] == "Wyborcza"]

lemma_wyborcza_new = lemma_wyborcza.merge(df_wyborcza[['id', 'date', 'eco_id', "id_new"]], how='inner', on=['id', 'date']).drop(columns=["Unnamed: 0"])

len(lemma_wyborcza) == len(lemma_wyborcza_new)

True

In [6]:
lemma_wyborcza_new['id'] = lemma_wyborcza_new['id_new']
lemma_wyborcza_new = lemma_wyborcza_new.drop(columns=["id_new"])

In [7]:
lemma = lemma.drop(columns=["Unnamed: 0"])
lemma_other = lemma[lemma['source'] != "Wyborcza"]

In [8]:
total_articles = 0
data = []
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        df = pd.read_csv(os.path.join(directory, filename))
        print(filename)
        print(len(df))
        total_articles += len(df)
        data.append(df)

data = pd.concat(data)
data_other = data[data['source'] != "Wyborcza"]

eco_gpc.csv
3874
eco_wpolityce.csv
498
eco_newsweek.csv
645
eco_dorzeczy.csv
307
eco_polityka.csv
950
eco_wprost.csv
538
eco_wyborcza.csv
6241
eco_rzepa.csv
6933


In [9]:
lemma_other_new = lemma_other.merge(data_other[['source', 'id', 'eco_id']], on=['source', 'id'], how='left')

In [ ]:
lemma_wyborcza_new['eco_id'] = lemma_wyborcza_new['eco_id'].astype('Int64')
lemma_other_new['eco_id'] = lemma_other_new['eco_id'].astype('Int64')

In [11]:
lemma_combined = pd.concat([lemma_other_new, lemma_wyborcza_new])

In [13]:
lemma_combined.to_csv("lemma_new.csv", index=False)

In [1]:
from combo.predict import COMBO

model = COMBO.from_pretrained('polish-herbert-base-ud213')

prediction = model("Czołem! - przywitał się Staś. Odwrócił się na pięcie i wyszedł. Chciałabym, żebyśmy poszli na spacer - powiedziała dziewczyna.")

print("{:15} {:15} {:10} {:3} {:10}".format('TOKEN', 'LEMMA', 'UPOS', 'HEAD', 'DEPREL'))
for i, sentence in enumerate(prediction):
  print('======= SENTENCE {} ======='.format(str(i+1)))
  for token in sentence.tokens:
      print("{:15} {:15} {:10} {:3} {:10}".format(token.text, token.lemma, token.upostag, token.head, token.deprel))

/home/ps/miniconda3/envs/combo/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using model LAMBO_213-UD_Polish-PDB
TOKEN           LEMMA           UPOS       HEAD DEPREL    
======= SENTENCE 1 =======
Czołem          czół            PROPN        4 parataxis:obj
!               !               PUNCT        1 punct     
-               -               PUNCT        4 punct     
przywitał       przywitać       VERB         0 root      
się             się             PRON         4 expl:pv   
Staś            Staś            PROPN        4 nsubj     
.               .               PUNCT        4 punct     
======= SENTENCE 2 =======
Odwrócił        odwrócić        VERB         0 root      
się             się             PRON         1 expl:pv   
na              na              ADP          4 case      
pięcie          pięta           NOUN         1 obl       
i               i               CCONJ        6 cc        
wyszedł         wyjść           VERB         1 conj      
.               .               PUNCT        1 punct     
======= SENTENCE 3 =======
Chciała  

In [2]:
import os
import pandas as pd
data = []
for file in os.listdir("corpus"):
    df = pd.read_csv("corpus/" + file, usecols=['id', 'source', 'date', 'text'])
    data.append(df)

data = pd.concat(data)

In [3]:
df.isnull().mean()

source    0.0
id        0.0
date      0.0
text      0.0
dtype: float64

In [5]:
def combo_lemmatize(text):
    prediction = model(text)
    lemmas = []
    for sentence in prediction:
        for token in sentence.tokens:
            lemmas.append(token.lemma)
    return " ".join(lemmas)

texts = df.text.iloc[:5]
for text in texts:
    lemma = combo_lemmatize(text)